# Example for generating photoionization models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyCloudy as pc

from sys import path
path.insert(0,'../CloudyGalaxy/')

from gas_stats import GasStatsBy17 as GasStats
from abundances import ElementalAbundances, convert_xi_to_F
from read_write_cloudy import StellarSpectrum, format_emission_table, make_input_file, make_emission_line_files

### Setting the data paths and cores to use for Cloudy

In [2]:
n_proc              = 6

output_dir          = './output_data/'
model_name          = 'test_model_v5'
data_path           = '../CloudyGalaxy/input_data/'
star_table_filename = 'Chabrier_constant_SFH_IMF_lower_0p08_IMF_upper_120_star_table.ASCII'
emission_line_list  = format_emission_table('../CloudyGalaxy/input_data/cloudyLines_2021-03-29.dat')

### Defining the grid of free parameters for which to produce models

In [3]:
nlogZs = 15
nlogUs = 2
nxis   = 2
ntaus  = 4

logZs = np.linspace(-1.0, 0.7, nlogZs)
logUs = np.array([-3.0, -2.5])#np.linspace(-3.5, -1.5, nlogUs)
xis   = np.array([0.2, 0.35])#np.linspace(0.1, 0.6, nxis)
taus  = np.linspace(0.01, 4.0, ntaus)
Fs    = np.ones((nlogZs,nxis))*np.nan

### Making the Cloudy input files

In [4]:
pc.print_make_file(dir_ = output_dir)
for i in range(nlogZs):
    logZ = logZs[i]
    xi_to_F = convert_xi_to_F(logZ)
    for j in range(nlogUs):
        for k in range(nxis):
            logU = logUs[j]
            xi = xis[k]
            F = xi_to_F(xi)
            Fs[i,k] = F
            stellar_spectrum = StellarSpectrum(data_path, star_table_filename)
            gas_stats = GasStats(stellar_spectrum, logZ, logU)
            gas_stats.calc_all_parameters()
            elemental_abundances = ElementalAbundances(logZ, F)
            elemental_abundances.calc_all_parameters()
            make_input_file(output_dir, model_name, logZ, logU, xi, emission_line_list, stellar_spectrum, gas_stats, elemental_abundances)


### Running Cloudy

In [5]:
pc.run_cloudy(dir_ = output_dir, n_proc = n_proc, model_name = model_name, use_make = True)

### Extracting the emission line luminosities from the Cloudy data files and storing them in .npy files

In [6]:
make_emission_line_files(output_dir, model_name, logZs, logUs, xis, taus, Fs)

./output_data/test_model_v5_-1.000e+00_-3.000e+00_2.000e-01
1 models found!
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.2000, tau = 0.0100.
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.2000, tau = 1.3400.
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.2000, tau = 2.6700.
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.2000, tau = 4.0000.
./output_data/test_model_v5_-1.000e+00_-3.000e+00_3.500e-01
1 models found!
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.3500, tau = 0.0100.
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.3500, tau = 1.3400.
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.3500, tau = 2.6700.
PROCESSING: logZ = -1.0000, logU = -3.0000, xi = 0.3500, tau = 4.0000.
./output_data/test_model_v5_-1.000e+00_-2.500e+00_2.000e-01
1 models found!
PROCESSING: logZ = -1.0000, logU = -2.5000, xi = 0.2000, tau = 0.0100.
PROCESSING: logZ = -1.0000, logU = -2.5000, xi = 0.2000, tau = 1.3400.
PROCESSING: logZ = -1.0000, logU = -2.5000, xi = 0.2000, tau =

1 models found!
PROCESSING: logZ = -0.3929, logU = -2.5000, xi = 0.3500, tau = 0.0100.
PROCESSING: logZ = -0.3929, logU = -2.5000, xi = 0.3500, tau = 1.3400.
PROCESSING: logZ = -0.3929, logU = -2.5000, xi = 0.3500, tau = 2.6700.
PROCESSING: logZ = -0.3929, logU = -2.5000, xi = 0.3500, tau = 4.0000.
./output_data/test_model_v5_-2.714e-01_-3.000e+00_2.000e-01
1 models found!
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.2000, tau = 0.0100.
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.2000, tau = 1.3400.
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.2000, tau = 2.6700.
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.2000, tau = 4.0000.
./output_data/test_model_v5_-2.714e-01_-3.000e+00_3.500e-01
1 models found!
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.3500, tau = 0.0100.
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.3500, tau = 1.3400.
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.3500, tau = 2.6700.
PROCESSING: logZ = -0.2714, logU = -3.0000, xi = 0.

1 models found!
PROCESSING: logZ = 0.3357, logU = -2.5000, xi = 0.3500, tau = 0.0100.
PROCESSING: logZ = 0.3357, logU = -2.5000, xi = 0.3500, tau = 1.3400.
PROCESSING: logZ = 0.3357, logU = -2.5000, xi = 0.3500, tau = 2.6700.
PROCESSING: logZ = 0.3357, logU = -2.5000, xi = 0.3500, tau = 4.0000.
./output_data/test_model_v5_4.571e-01_-3.000e+00_2.000e-01
1 models found!
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.2000, tau = 0.0100.
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.2000, tau = 1.3400.
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.2000, tau = 2.6700.
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.2000, tau = 4.0000.
./output_data/test_model_v5_4.571e-01_-3.000e+00_3.500e-01
1 models found!
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.3500, tau = 0.0100.
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.3500, tau = 1.3400.
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.3500, tau = 2.6700.
PROCESSING: logZ = 0.4571, logU = -3.0000, xi = 0.3500, tau = 4.